# Promises and Async Functions

A good article: https://itnext.io/javascript-promises-and-async-await-as-fast-as-possible-d7c8c8ff0abc

* JavaScript executes in a single threaded process (Web Workers run in seperate processes)
* A single thread executes in sequence, not in parallel (contrast with threads in C# or Java)
* Asynchronous JavaScript mechanisms:
    - Asynchronous callbacks (```addEventListener```, ```setInterval```, ```setTimeout```, ```XMLHttpRequest```, etc.)
    - Asynchronous ```Promise``` objects
    - The ```async```/```await``` Keywords

* Asynchronous Callbacks
    - One way of doing async programming is to provide a callback function (called back on completion)
    - Asynchronous operations that depend on each other must be nested
    - Callback nesting can get messy ("callback hell")

    ```javascript
    someAsyncOperation(someParams, someCallback); // calls provided callback function
    function someCallback(someResult) {
        // Do something when called back
    }
    ```
    
* Asynchronous Promises
    - Allows asynchronous operations that depend on each other without "callback hell"
    - Instead of nesting callbacks in callbacks in callbacks, you just chain .then() calls
    - A Promise object wraps an asynchronous operation and notifies when it is done
    - Promise object represents eventual completion (or failure) of an asynchronous operation
    - Promise object also provides access to the resulting value of the asynchronous operation
    - Promise provides ```then()``` for success result and ```catch()``` for failure result
    - Promise states: ```pending``` (initial), ```fulfilled``` (success), ```rejected``` (failed)
    
    ```javascript
    someAsyncOperation(someParams) // returns a Promise object
    .then(function(result) {
        // Do something with success result
    })
    .catch(function(error) {
        // Handle error on failure result
    });
    ```
    
* The ```async```/```await``` Keywords
    - Async/Await is a cleaner syntax for handling multiple nested promises in synchronous-looking manner
    - Async/Await is a language feature that is a part of the ES8 standard (Node.js 7.6)
    - Async function is impemented with ```async``` keyword and called with ```await``` keyword
    - The ```await``` keyword can only be used inside a function marked as ```async```
    
    ```javascript
    async function getSomeAsyncData(value){
        const result = await fetchTheData(someUrl, value);
        return result;
    }
    ```

### Example Asyncronous Callback Hell

* See: https://developers.google.com/web/ilt/pwa/working-with-promises
* See: https://developers.google.com/web/ilt/pwa/lab-promises

```javascript
function isUserTooYoung(id, callback) {
  openDatabase(function(db) {
    getCollection(db, 'users', function(col) {
      find(col, {'id': id}, function(result) {
        result.filter(function(user) {
          callback(user.age < cutoffAge);
        });
      });
    });
  });
}
```

### Example Using Promises

```javascript
function isUserTooYoung(id) {
  return openDatabase() // returns a promise
  .then(function(db) {return getCollection(db, 'users');})
  .then(function(col) {return find(col, {'id': id});})
  .then(function(user) {return user.age < cutoffAge;});
}
```

### Example of ```async``` and ```await``` Keywords

See: [Refactoring to async/await](https://advancedweb.hu/how-to-refactor-a-promise-chain-to-async-functions)

```javascript
async function isUserTooYoung(id) {
    let db = await openDatabase();
    let col = await ((db) => getCollection(db, 'users'));
    let user = await ((col) => find(col, {'id': id}));
    let result = await ((user) => user.age < cutoffAge);
    return result;
}
```

## Simple Asynchronous Callbacks

* Conceptually simplest asynchronous programming technique
* Temporal chain of dependencies implemented using nesting callbacks
* Callback hell results when temporal chain of callback dependencies is long

In [1]:
{ // using setTimeout to simulate asynchronous task (like waiting for UI event)
function callBack(result) {
    console.log("callBack result:", result);
}
function scheduleCallback() {
    console.log("entered scheduleCallback()");
    setTimeout(function() {
        callBack(Math.random());
    }, 3000);
}
console.log("about to call scheduleCallback()");
scheduleCallback();
console.log("meanwhile... carrying on with business");
}

about to call scheduleCallback()
entered scheduleCallback()
meanwhile... carrying on with business
callBack result: 0.8296379981758293


## Promises

A Promise object wraps the eventual completion (or failure) of an asynchronous operation, and its final resulting value.

* Solves callback hell by chaining promises rather than nesting callbacks
* A bot harder to understand than simple callbacks
* Promise provides then() for success result and catch() for failure result
* Promise can only be in one of three states at any given time:
    - pending (initial)
    - fulfilled (success)
    - rejected (failed)
* Promises can be chained to represent a temporal sequence of dependancies

In [2]:
{ // an example that can only succeed...
var myPromise = new Promise(function(resolve, reject) {
    setTimeout(resolve, 1000, 'Our work here is done!');
});

console.log("Starting:", myPromise); // output: Promise { <pending> }

myPromise.then(function(successResult) { // set up callback for eventual result
    console.log("Success:", successResult);
});

console.log("Pending:", myPromise); // Promise { <pending> }

var counter = 0;
var id = setInterval(function() {
    counter++;
    if (counter > 15)
        clearInterval(id);
    else
        console.log(counter, myPromise, 'busy with other stuff');
}, 100)
}

Starting: Promise { <pending> }
Pending: Promise { <pending> }
1 Promise { <pending> } 'busy with other stuff'
2 Promise { <pending> } 'busy with other stuff'
3 Promise { <pending> } 'busy with other stuff'
4 Promise { <pending> } 'busy with other stuff'
5 Promise { <pending> } 'busy with other stuff'
6 Promise { <pending> } 'busy with other stuff'
7 Promise { <pending> } 'busy with other stuff'
8 Promise { <pending> } 'busy with other stuff'
9 Promise { <pending> } 'busy with other stuff'
Success: Our work here is done!
10 Promise { 'Our work here is done!' } 'busy with other stuff'
11 Promise { 'Our work here is done!' } 'busy with other stuff'
12 Promise { 'Our work here is done!' } 'busy with other stuff'
13 Promise { 'Our work here is done!' } 'busy with other stuff'
14 Promise { 'Our work here is done!' } 'busy with other stuff'
15 Promise { 'Our work here is done!' } 'busy with other stuff'


In [4]:
{ // an example that can succeed or fail randomly (try running several times)
var myPromise = new Promise(function(resolve, reject) {
    if (Math.random() > 0.5) { // toss a 50/50 coin
        setTimeout(resolve, 1000, 'Our work here is done!');
    }
    else {
        setTimeout(reject, 1000, 'Ooops!');
    }
});

console.log("Starting:", myPromise); // output: Promise { <pending> }

myPromise.then(function(successResult) { // set up callback for eventual result
    console.log("Success:", successResult);
});

console.log("Pending:", myPromise); // Promise { <pending> }

var counter = 0;
var id = setInterval(function() {
    counter++;
    if (counter > 15)
        clearInterval(id);
    else
        console.log(counter, myPromise, 'busy with other stuff');
}, 100)
}

Starting: Promise { <pending> }
Pending: Promise { <pending> }
1 Promise { <pending> } 'busy with other stuff'
2 Promise { <pending> } 'busy with other stuff'
3 Promise { <pending> } 'busy with other stuff'
4 Promise { <pending> } 'busy with other stuff'
5 Promise { <pending> } 'busy with other stuff'
6 Promise { <pending> } 'busy with other stuff'
7 Promise { <pending> } 'busy with other stuff'
8 Promise { <pending> } 'busy with other stuff'
9 Promise { <pending> } 'busy with other stuff'
10 Promise { <rejected> 'Ooops!' } 'busy with other stuff'


(node:9664) UnhandledPromiseRejectionWarning: Ooops!
(node:9664) UnhandledPromiseRejectionWarning: Unhandled promise rejection. This error originated either by throwing inside of an async function without a catch block, or by rejecting a promise which was not handled with .catch(). (rejection id: 1)
(node:9664) [DEP0018] DeprecationWarning: Unhandled promise rejections are deprecated. In the future, promise rejections that are not handled will terminate the Node.js process with a non-zero exit code.


11 Promise { <rejected> 'Ooops!' } 'busy with other stuff'
12 Promise { <rejected> 'Ooops!' } 'busy with other stuff'
13 Promise { <rejected> 'Ooops!' } 'busy with other stuff'
14 Promise { <rejected> 'Ooops!' } 'busy with other stuff'
15 Promise { <rejected> 'Ooops!' } 'busy with other stuff'


## The ```async``` and ```await``` Keywords

* The ```async``` and ```await``` keywords support promises using a simplified syntax
* An ```async``` before a function means that it returns a promise rather than an immediate value
* The ```await``` keyword is only permitted inside async functions
* The ```await``` keyword makes the fucntion wait until promise reolves and then returns the result
* If the function rejects the promise it throws an error
* If the function rejects the promise we can append ```.catch()``` to handle it

In [2]:
{
async function f() {
    return 1;
}
console.log(f());                                     // Promise { 1 }
f().then(result => console.log("then: " +  result));  // then: 1
}

Promise { 1 }
then: 1


In [3]:
{
async function f() {
  let promise = new Promise((resolve, reject) => {
    setTimeout(() => resolve("resolve: done!"), 1000)
  });
  let result = await promise; // wait until the promise resolves
  console.log(result);        // resolve: done! (after one second delay)
}
f();
}

resolve: done!


In [4]:
{
async function f() {
    await Promise.reject(new Error("oops!"));        // same as -> throw new Error("oops!");
}
f().then(result => console.log("then: " + result))
    .catch(err => console.log("catch: " + err));     // catch: Error: oops!
}

catch: Error: oops!


In [6]:
async function wait() {
    await new Promise(resolve => setTimeout(resolve, 1000));
    return 10;
}
function f() {
    wait().then(result => console.log(result)); // 10 (after 1 second delay)
}
f();

10
